##### This is the offline version of the MARS_LANDER capable of simple vertical descent only

In [ ]:
import neat
import math 
import numpy as np
#defining macros
MARS_RADIUS = 3386000.0 # (m)
MARS_MASS = 6.42E23 # (kg)
GRAVITY = 6.673E-11 # (m^3/kg/s^2)
MARS_DAY = 88642.65 # (s)
EXOSPHERE = 200000.0 # (m)

# Lander constants
LANDER_SIZE = 1.0 # (m)
UNLOADED_LANDER_MASS = 100.0 # (kg)
FUEL_CAPACITY = 100.0 #(l)
FUEL_RATE_AT_MAX_THRUST = 0.5 #(l/s)
FUEL_DENSITY = 1.0 #(kg/l)

MAX_THRUST = (1.5 * (FUEL_DENSITY*FUEL_CAPACITY+UNLOADED_LANDER_MASS) * (GRAVITY*MARS_MASS/(MARS_RADIUS*MARS_RADIUS))) # (N)
ENGINE_LAG = 0.0 # (s)
ENGINE_DELAY = 0.0 # (s)
DRAG_COEF_CHUTE = 2.0
DRAG_COEF_LANDER = 1.0
MAX_PARACHUTE_DRAG = 20000.0 # (N)
MAX_PARACHUTE_SPEED = 500.0 # (m/s)
THROTTLE_GRANULARITY = 20 # for manual control
MAX_IMPACT_GROUND_SPEED = 1.0 # (m/s)
MAX_IMPACT_DESCENT_RATE = 1.0 # (m/s)
UNIVERSAL_GRAVITATION_CONSTANT = 6.643E-11
ORBITAL_VELOCITY = 3500 #(m/s)
SMALL_NUM= 0.0000001

DELTA_T = 0.1 #(s)

#defining variables

The run simulation method of the lander class below serves the purpose of a minimisation function

In [ ]:
class lander:
    
    def __init__( self, kh, kp, thresh_hold, height = 10000):
        
        self.position = np.array([0.0, -(MARS_RADIUS + height), 0.0])
        self.prev_position = self.position
        self.velocity = np.array([0, 0 ,0])
        self.orientation = np.array([0,0,0])
        
    
        self.fuel = 1 #(L)
        self.simulation_time = 0
        self.throttle = 0
        self.altitude = np.linalg.norm(self.position) - MARS_RADIUS
        
        self.parachute_deployed = "NOT_DEPLOYED"
        self.crashed = False
        self.landed = False   

        self.kh = kh
        self.kp = kp      
        self.thresh_hold = thresh_hold


# Uses orientation
    def thrust_wrt_world(self):
        return self.throttle*MAX_THRUST*self.position / np.linalg.norm(self.position) 
        
        
    def atmospheric_density(self):        
        alt = np.linalg.norm(self.position) - MARS_RADIUS
        if ((alt > EXOSPHERE) or (alt < 0.0)) :
            return 0.0
        else :
            return (0.017 * math.e**(-alt/11000.0))   
      
    def safe_to_deploy_parachute (self):
        
        drag = 0.5*DRAG_COEF_CHUTE*self.atmospheric_density()*5.0*2.0*LANDER_SIZE*2.0*LANDER_SIZE*np.linalg.norm(self.velocity)**2
        # Do not use the global variable "altitude" here, in case this function is called from within the
        # numerical_dynamics function, before altitude is updated in the update_visualization function
        if ((drag > MAX_PARACHUTE_DRAG) or ((np.linalg.norm(self.velocity) > MAX_PARACHUTE_SPEED) and ((np.linalg.norm(self.position) - MARS_RADIUS) < EXOSPHERE))):
            return False
        else:
            return True
 
    def numerical_dynimics(self):
        
        thrust_force = self.thrust_wrt_world()           
        gravitational_acc = self.position * (( - MARS_MASS * UNIVERSAL_GRAVITATION_CONSTANT) / (np.linalg.norm(self.position)**(3)))
        atm_density = self.atmospheric_density()
        lander_drag = - 0.5 * atm_density * DRAG_COEF_LANDER *  3.14 * ( 2 * LANDER_SIZE)**2 * np.linalg.norm(self.velocity) * self.velocity
        chute_drag = - 0.5 * atm_density * DRAG_COEF_CHUTE * 5 * ( 2 * LANDER_SIZE)**2 * np.linalg.norm(self.velocity) * self.velocity
        mass_lander = UNLOADED_LANDER_MASS + FUEL_CAPACITY * FUEL_DENSITY * self.fuel
        
        
        if self.parachute_deployed == "NOT_DEPLOYED":
            acceleration = gravitational_acc + (lander_drag + thrust_force) / mass_lander            
        else:
            acceleration = gravitational_acc + (lander_drag + thrust_force + chute_drag) / mass_lander

       
        if  self.simulation_time == 0 :            
            self.prev_position = self.position
            self.position = self.position + self.velocity * DELTA_T            
        else:
            new_position = 2 * self.position - self.prev_position + (DELTA_T** 2) * acceleration
            self.prev_position = self.position
            self.position = new_position 
        self.autopilot()
       

    def autopilot(self):       

    
        if self.safe_to_deploy_parachute() == True:
            self.parachute_deployed ="DEPLOYED"
        else:
            self.parachute_deployed = "NOT_DEPLOYED"
        
        self.velocity = (self.position - self.prev_position)/DELTA_T
        error = -(0.5 + self.kh * self.altitude- np.linalg.norm(self.velocity)) 
        power = self.kp * error
        
        
        if  power < - self.thresh_hold :
            self.throttle = 0
        elif power < 1 - self.thresh_hold :                
            self.throttle = self.thresh_hold + power
        else:
            self.throttle = 1

                
    def updating_variables(self):
        
        self.altitude =  np.linalg.norm(self.position) - MARS_RADIUS        
        if self.altitude < LANDER_SIZE/2:
            
            if abs(np.linalg.norm(self.velocity)) > MAX_IMPACT_DESCENT_RATE :
                self.crashed = True
            self.landed = True

        self.fuel -= DELTA_T * (FUEL_RATE_AT_MAX_THRUST * self.throttle)/FUEL_CAPACITY
        
        if self.fuel <= 0.0:
            self.fuel = 0

        if self.landed or self.fuel == 0 :
            self.throttle = 0

        if (self.parachute_deployed == "DEPLOYED") :
            if (not self.safe_to_deploy_parachute()): 
                parachute_lost = True # to guard against the autopilot reinstating the parachute!
                self.parachute = "NOT_DEPLOYED"

    
    def run_simulation(self):

        while self.landed == False :
            self.numerical_dynimics()
            self.updating_variables()
            self.simulation_time += DELTA_T


        if np.linalg.norm(self.velocity) >= 1:
             return 1000
        else:
        # minimisation functions for vertical descent - change this function to the commented
        # ones - in case seachign for minimum fuel usage or minimum descent velocity 
            #return self.simulation_time
        
        #minimum fuel consumption 
             return(1 - self.fuel)
        #minimum descent rate
        #     return (np.linalg.norm(self.velocity))
                 



In [ ]:
lan = lander(0.06 ,0.99985194, 0.14017362) # enter values for kh, kp , threshhold and height(optional)
print(lan.run_simulation())
print(lan.velocity)

 Since our minimisation function is not necessarily continuous in the multidimensional space (here 3 dimensions coressponding to kh, kp, threshhold) as we just return 1000 in case the lander crashes, using algorithms that use gradient descent methode would probably be not the best idea. Therefore, to minimise the function we use Nelder - Mead/ Simplex Search algorithm, a direct search algorithm that does not use gradient information. 

##### A quick overview of Nelder-Mead algorithm

The method uses a simplex - a polytope of n + 1 vertices in n (no of input parameters) dimensional space. Eg. a triangle on a plane, a tetrahedron in three dimensional space etc. It then evaluates the optimisation function at n arbitrary points to begin with. The worst point is the point where the function evaluation is maximum. The worst point is then repaced by a new - better - point by a variety of techniques such as reflection, expansion , contraction of the polytope etc. This is done iteratively until the polytope converges and a minima is found. 


In [ ]:
from scipy.optimize import minimize
import numpy as np
from random import uniform

# specify height as a global variable and pass seprately in func, if you want to 
# find pid control values for an altitude that is different than 10km
# eg.
# height = 200000
# func = lambda x:  lander(x[0],x[1], x[2], height).run_simulation()

height = 200000
func = lambda x:  lander(x[0],x[1], x[2], height).run_simulation()
bounds = [(0.04,0.06 ), (0, 2), (0, 2)]

def f(x):
    a = minimize(func, [x[0], x[1], x[2]],  method='nelder-mead',
                           options={ 'disp': True})
    return a

# Call the function with a seed - and print the input values at which the optimisation function detects a minima
res = f([0.14260836, 1.50728959, 0.151973576])
print(res.x)  

Some comiplation of results 

1. Minimum Descent Velocity

    |kh|          kp|          threshhold|
    |--|-------------|--------------------|
    |0.03853176 |0.60199072 |0.80496405|



2. Minimum fuel Usage

Data for minimum fuel usage with the seed as 0.04260836, 0.90728959, 0.51973576

|kh        |kp          | threshhold| height|
|----------|------------|-----------|-------|
|0.06      | 0.99985194 |0.14017362 | 10000 |
|0.06      | 0.99980863 |0.14121843 | 20000 |
|0.06      | 0.99815977 |0.1395215  | 30000 |
|0.06      | 0.99962401 |0.13974803 | 40000 |
|0.06      | 0.99962659 |0.13999489 | 50000 |
|0.06      | 0.99533407 |0.13907706 | 60000 |
|0.06      | 0.99991921 |0.13676669 | 70000 |
|0.06      | 0.99818652 |0.13988179 | 80000 |
|0.06      | 0.9981079  |0.13879895 | 90000 |
|0.06      | 0.99949246 |0.1396833  | 100000|
|0.06      | 0.99972355 |0.1405565  | 110000|
|0.06      | 0.99986975 |0.14150887 | 120000|
|0.06      | 0.99976039 |0.14059086 | 130000|
|0.06      | 0.99997883 |0.1409362  | 140000|
|0.06      | 0.99985228 |0.14152659 | 150000|
|0.06      | 0.99815874 |0.1395263  | 160000|
|0.06      | 0.99552597 |0.13839891 | 170000|
|0.06      | 0.99992165 |0.14109578 | 180000|
|0.06      | 0.97057914 |0.14986541 | 200000|
|0.06      | 0.97916558 |0.14584812 | 400000|
|0.06      |         1. |0.13653684 | 600000|
|0.06      | 0.94773791 |0.16477681 | 800000|
|0.06      |     1.     |  0.136733 |1000000|
|0.06      | 0.95476514 |0.15732394 |1200000|
|0.06      | 0.95418197 |0.15329436 |1400000|
|0.06      | 0.99989855 |0.12871853 |1600000|
|0.06      | 0.91905964 |0.15989027 |1800000|
|0.06      | 0.99985838 |0.12147098 |2000000|
|0.06      | 0.99997476 |0.12141989 |2200000|
|0.06      | 1.         |0.11844587 |2400000|
|0.06      | 0.99997827 |0.11870772 |2600000|
|0.06      | 0.98568824 |0.12155331 |2800000|
|0.06      | 0.99997268 |0.11500476 |3000000|
|0.06      | 0.90376293 |0.15891031 |3200000|
|0.06      | 0.91675522 |0.1554941  |3400000|
|0.06      | 0.94579259 |0.13810358 |3600000|


3 . Minimum Descent time

Data for minimum descent time with the seed as 0.04260836, 0.90728959, 0.51973576


|Kh        | kp        | threshhold|height|
|----------|-----------|-----------|------|
|0.16497574| 1.82689211| 0.1045009 | 10000|      
|0.16487285| 1.67434604| 0.0426761 | 20000|
|0.16491779| 1.5380842 | 0.05154344| 30000|
|0.16484612| 1.55470999| 0.04902635| 40000|
|0.16484612| 1.55470999| 0.04902635| 50000|
|0.16503771| 1.76752234| 0.02930282| 60000|
|0.16503771| 1.76752234| 0.02930282| 70000|
|0.16488967| 1.53724793| 0.05164545| 80000|
|0.16490431| 1.55340662| 0.05055183| 90000|
|0.16475262| 1.54865889| 0.04937717| 100000|
|0.16502185| 1.81907146| 0.10480012| 110000|
|0.16491539| 1.67581538| 0.04256608| 120000|
|0.16502185| 1.81907146| 0.10480012| 130000|
|0.16487285| 1.67434604| 0.0426761 | 140000|
|0.16491539| 1.67581538| 0.04256608| 150000|
|0.16491779| 1.5380842 | 0.05154344| 160000|
|0.12365394| 0.92693811| 0.24544351| 170000|
|0.16487285| 1.67434604| 0.0426761 | 180000|

These values of kh, kp, threshhold from the minimisation function are very specific to the initial heights. However, to make only one set of kh, kp, threshhold values suitable for each autopilot, I used a small tweak.
If one observes carefully the values of kh, kp, and threshhold one can analyse a general trend with height in each case. 
In almost all cases, the problem with using (kh,kp, threshhold) value set corresponding to a different height is that the velocity becomes 0 a little earlier when the lander has still not landed.
The algorithm continues to run and velocity changes direction and the lander starts climbing up. Thus I have added another functionality in the pid control for descent that when the if the climb rate becomes positive, the throttle is assigned 0 and the lander is allowed to fall under gravity.
   

My choice of final sets of values for the three autopilots were based after many trials (- with each trial using a different seed to begin with ) and after observing a generalised range of values over different heights.   